<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/sac%20trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!pip install tf_agents[reverb]
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import pandas as pd
import numpy as np

import pandas_ta as ta

from tf_agents.environments import suite_gym
import tensorflow as tf

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec
from tf_agents.environments import utils

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

In [141]:
# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 10 # @param {type:"integer"}

num_eval_episodes = 5 # @param {type:"integer"}
eval_interval = 1000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

In [79]:
df = pd.read_csv('/content/drive/MyDrive/eurusd_hour.csv')
bid_open = df['BO'].to_numpy()
bid_close = df['BC'].to_numpy()
rel_change = (bid_open - bid_close) * 1000
input_df = pd.DataFrame()

cumulative_arr = np.cumsum(rel_change)
rsi = ta.rsi(df['BC'], 14)
stdev = ta.stdev(df['BC'], 14)
mad = ta.sma(df['BC'], 14) - df['BC']
mad = pd.DataFrame(mad,columns=["mad"])
relc_df = pd.DataFrame(rel_change,columns=["rel_change"])

concatenated_df = pd.concat([rsi, stdev, mad,relc_df], axis=1)
concatenated_df = concatenated_df.dropna(axis=0)

changenp = concatenated_df.pop("rel_change")
changenp = changenp.to_numpy()
changenp = changenp.astype('float32')

concatenated_df = concatenated_df.to_numpy()
concatenated_df = concatenated_df.astype('float32')

print(concatenated_df[1])

[5.2937088e+01 8.4042799e-04 6.5000000e-05]


In [150]:
new_array = np.append(concatenated_df[0], 20.)
print(new_array)

[3.60771217e+01 9.35611664e-04 1.96000002e-03 2.00000000e+01]


In [197]:
class CardGameEnv(py_environment.PyEnvironment):
    def __init__(self):
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(1,), dtype=np.float32, minimum=-1, maximum=1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(4,), dtype=np.float32, minimum=-100, maximum=100, name='observation')
        self._array = concatenated_df
        self._state = np.append(self._array[0],0)
        self._episode_ended = False
        self._count = 0
        self._Net_Profit = 0
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._Net_Profit = 0
        self._count = 0
        self._state = np.append(self._array[0],0)
        self._episode_ended = False
        return ts.restart(np.array(self._state, dtype=np.float32))

    def _step(self, action):
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()

        value = changenp[self._count]

        # Make sure episodes don't go on forever.

        self._Net_Profit += value*action[0]
        self.state = np.append(self._array[self._count],self._Net_Profit)

        if self._count > 5000:
            self._episode_ended = True
        self._count += 1

        if self._episode_ended:
            reward = self._Net_Profit
            return ts.termination(np.array(self._state, dtype=np.float32), reward)
        else:
            reward = value*action[0]
            return ts.transition(np.array(self._state, dtype=np.float32), reward=reward,discount=.9)

In [198]:

environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=2)
#env = environment#tf_py_environment.TFPyEnvironment(environment)

In [199]:
env = environment #tf_py_environment.TFPyEnvironment(environment)
env_name = 'MountainCarContinuous-v0'
collect_env  = env #suite_gym.load(env_name)
eval_env   = env #suite_gym.load(env_name)

In [200]:
use_gpu = False

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

In [201]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=(20,20),
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [202]:
print("Observation Spec:", observation_spec)
print("Action Spec:", action_spec)

Observation Spec: BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array(-100., dtype=float32), maximum=array(100., dtype=float32))
Action Spec: BoundedTensorSpec(shape=(1,), dtype=tf.float32, name='action', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32))


In [203]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=(20,20),
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [204]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.keras.optimizers.Adam(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.keras.optimizers.Adam(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

In [205]:
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)

table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

In [206]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

In [207]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

In [208]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

In [209]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

In [210]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

In [211]:
location = "/content/drive/MyDrive/lerning"

In [212]:

env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(location, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

In [213]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(location, 'eval'),
)

In [214]:
saved_model_dir = os.path.join(location, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  location,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers,
  strategy=strategy)

In [215]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [216]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

step = 0: AverageReturn = 160.809921, AverageEpisodeLength = 5002.000000


In [ ]:
# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

step = 10: loss = 117.2213134765625
step = 20: loss = 60.76483154296875
step = 30: loss = 27.69717025756836
step = 40: loss = 16.217378616333008
step = 50: loss = 11.839845657348633
step = 60: loss = 11.779899597167969
step = 70: loss = 7.810325622558594
step = 80: loss = 4.919992446899414
step = 90: loss = 3.222858428955078
step = 100: loss = 2.0979723930358887
step = 110: loss = 1.543143630027771
step = 120: loss = 0.8599667549133301
step = 130: loss = 0.727506160736084
step = 140: loss = 0.6304185390472412
step = 150: loss = 0.543962836265564
step = 160: loss = 0.20781072974205017
step = 170: loss = 0.3304329514503479
step = 180: loss = 0.10754910111427307
step = 190: loss = 0.12839704751968384
step = 200: loss = 0.03825414180755615
step = 210: loss = 0.039260268211364746
step = 220: loss = -0.017891645431518555
step = 230: loss = -0.008063852787017822
step = 240: loss = -0.17437812685966492
step = 250: loss = -0.027187764644622803
step = 260: loss = -0.023187339305877686
step = 270